<center><a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a></center>

# 3. 卷積神經網路(Convolutional Neural Networks)

在前一節中，我們建立並訓練了一個簡單的模型來分類美國手語(ASL)圖像。該模型能夠以非常高的準確度(accuracy)正確地學習分類訓練資料集，但它在驗證(Validation)資料集上的表現卻不那麼好。這種無法很好地泛化到非訓練資料的行為稱為過度擬合([overfitting](https://scikit-learn.org/stable/auto_examples/model_selection/plot_underfitting_overfitting.html))，在本節中，我們將介紹一種稱為卷積神經網路([convolutional neural network](https://towardsdatascience.com/a-comprehensive-guide-to-convolutional-neural-networks-the-eli5-way-3bd2b1164a53))的常見模型，它特別擅長讀取圖像並對其進行分類。

## 3.1 目標(Objectives)

* 專門為卷積神經網路(CNN)準備資料
* 創建一個更複雜的卷積神經網路(CNN)模型，了解更多種類的模型層
* 訓練卷積神經網路(CNN)模型並觀察其性能

In [1]:
import torch.nn as nn
import pandas as pd
import torch
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

True

## 3.2 載入和準備資料

### 3.2.1 準備圖像


讓我們像在前一個課程練習(Lab)中那樣載入我們的 DataFrames：

In [2]:
train_df = pd.read_csv("data/asl_data/sign_mnist_train.csv")
valid_df = pd.read_csv("data/asl_data/sign_mnist_valid.csv")


這個美國手語(ASL)資料已經事先被扁平化了。

In [3]:
sample_df = train_df.head().copy()  # Grab the top 5 rows
sample_df.pop('label')
sample_x = sample_df.values
sample_x

array([[107, 118, 127, ..., 204, 203, 202],
       [155, 157, 156, ..., 103, 135, 149],
       [187, 188, 188, ..., 195, 194, 195],
       [211, 211, 212, ..., 222, 229, 163],
       [164, 167, 170, ..., 163, 164, 179]])

In [4]:
sample_x.shape

(5, 784)


在這種格式下，我們沒有關於哪些像素彼此相鄰的所有資訊。因此，我們無法應用能夠檢測特徵的卷積。讓我們重塑([reshape](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html))我們的資料集，使其採用28x28像素的格式。這將使我們的卷積能夠關聯像素組並檢測重要特徵。

請注意，對於我們模型的第一個卷積層，我們不僅需要圖像的高度和寬度，還需要顏色通道([color channels](https://www.photoshopessentials.com/essentials/rgb/))的數量。我們的圖像是灰階的，所以我們只有1個通道(channel)。

這意味著我們需要將目前的形狀 `(5, 784)` 轉換為 `(5, 1, 28, 28)`。使用 [NumPy](https://numpy.org/doc/stable/index.html) 陣列，我們可以為任何我們希望保持不變的維度使用`-1`作為設定值。

In [5]:
IMG_HEIGHT = 28
IMG_WIDTH = 28
IMG_CHS = 1

sample_x = sample_x.reshape(-1, IMG_CHS, IMG_HEIGHT, IMG_WIDTH)
sample_x.shape

(5, 1, 28, 28)

### 3.2.2 創建資料集

讓我們將上述步驟添加到我們的 `MyDataset` 類中。

#### 實作練習


下面的Class定義中有4個 `FIXME`。你能用正確的值替換它們嗎？

In [6]:
class MyDataset(Dataset):
    def __init__(self, base_df):
        x_df = base_df.copy()  # Some operations below are in-place
        y_df = x_df.pop(FIXME)
        x_df = x_df.values / 255  # Normalize values from 0 to 1
        x_df = x_df.reshape(-1, FIXME, FIXME, FIXME)
        self.xs = torch.tensor(x_df).float().to(device)
        self.ys = torch.tensor(y_df).to(device)

    def __getitem__(self, idx):
        x = self.xs[idx]
        y = self.ys[idx]
        return x, y

    def __len__(self):
        return len(self.xs)

#### 解答


點擊下方的 `...` 查看解答。

In [7]:
# SOLUTION
class MyDataset(Dataset):
    def __init__(self, base_df):
        x_df = base_df.copy()  # Some operations below are in-place
        y_df = x_df.pop('label')
        x_df = x_df.values / 255  # Normalize values from 0 to 1
        x_df = x_df.reshape(-1, IMG_CHS, IMG_WIDTH, IMG_HEIGHT)
        self.xs = torch.tensor(x_df).float().to(device)
        self.ys = torch.tensor(y_df).to(device)

    def __getitem__(self, idx):
        x = self.xs[idx]
        y = self.ys[idx]
        return x, y

    def __len__(self):
        return len(self.xs)

### 3.2.3 建立資料載入器(DataLoader)


接下來，讓我們從資料集(Dataset)建立資料載入器(DataLoader)

#### 實作練習


以下其中一個函式呼叫缺少了 `shuffle=True` 參數。你能記得是哪一個並將它加回去嗎？

In [8]:
BATCH_SIZE = 32

train_data = MyDataset(train_df)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE)
train_N = len(train_loader.dataset)

valid_data = MyDataset(valid_df)
valid_loader = DataLoader(valid_data, batch_size=BATCH_SIZE)
valid_N = len(valid_loader.dataset)

#### 解答

點擊下方的 `...` 查看解答。

In [9]:
# SOLUTION
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)


讓我們從資料載入器(DataLoader)中取得一個批次(batch)來確保它正常運作。

In [10]:
batch = next(iter(train_loader))
batch

[tensor([[[[0.6275, 0.6275, 0.6392,  ..., 0.6157, 0.6118, 0.6039],
           [0.6353, 0.6314, 0.6431,  ..., 0.6235, 0.6157, 0.6078],
           [0.6392, 0.6392, 0.6471,  ..., 0.6196, 0.6118, 0.6078],
           ...,
           [0.4627, 0.6784, 0.6706,  ..., 0.6706, 0.6667, 0.6784],
           [0.6510, 0.6627, 0.6784,  ..., 0.6784, 0.6549, 0.5882],
           [0.6157, 0.6588, 0.6706,  ..., 0.2863, 0.1961, 0.1098]]],
 
 
         [[[0.2196, 0.2431, 0.2941,  ..., 0.5333, 0.5333, 0.5373],
           [0.2196, 0.2471, 0.3020,  ..., 0.5333, 0.5373, 0.5412],
           [0.2235, 0.2549, 0.3098,  ..., 0.5373, 0.5373, 0.5451],
           ...,
           [0.3333, 0.3686, 0.3922,  ..., 0.6353, 0.6353, 0.6392],
           [0.3373, 0.3725, 0.3961,  ..., 0.6431, 0.6431, 0.6431],
           [0.3373, 0.3686, 0.3961,  ..., 0.6431, 0.6431, 0.6431]]],
 
 
         [[[0.4275, 0.4784, 0.5176,  ..., 0.6078, 0.6000, 0.5922],
           [0.4314, 0.4824, 0.5255,  ..., 0.6196, 0.6118, 0.6000],
           [0.4431


它看起來不同，但讓我們檢查 `shape` 來確認。

In [11]:
batch[0].shape

torch.Size([32, 1, 28, 28])

In [12]:
batch[1].shape

torch.Size([32])

## 3.3 建立卷積模型(Convolutional Model)


現今，許多資料科學家會從類似的專案借用其模型特性做為基礎來開始他們的專案。假設問題不是獨一無二的，那很有可能有人已經建立了表現良好的模型，並發布在如 [TensorFlow Hub](https://www.tensorflow.org/hub) 和 [NGC Catalog](https://ngc.nvidia.com/catalog/models) 等線上資源庫中。今天，我們將提供一個適合這個問題的模型。

<img src="images/cnn.png" width=180 />

我們在課程中介紹了許多不同類型的層，在這裡我們將全部介紹並提供它們的文件連結。當有疑問時，請閱讀官方文件（或在[Stack Overflow](https://stackoverflow.com/)上提問）。

In [13]:
n_classes = 24
kernel_size = 3
flattened_img_size = 75 * 3 * 3

model = nn.Sequential(
    # First convolution
    nn.Conv2d(IMG_CHS, 25, kernel_size, stride=1, padding=1),  # 25 x 28 x 28
    nn.BatchNorm2d(25),
    nn.ReLU(),
    nn.MaxPool2d(2, stride=2),  # 25 x 14 x 14
    # Second convolution
    nn.Conv2d(25, 50, kernel_size, stride=1, padding=1),  # 50 x 14 x 14
    nn.BatchNorm2d(50),
    nn.ReLU(),
    nn.Dropout(.2),
    nn.MaxPool2d(2, stride=2),  # 50 x 7 x 7
    # Third convolution
    nn.Conv2d(50, 75, kernel_size, stride=1, padding=1),  # 75 x 7 x 7
    nn.BatchNorm2d(75),
    nn.ReLU(),
    nn.MaxPool2d(2, stride=2),  # 75 x 3 x 3
    # Flatten to Dense
    nn.Flatten(),
    nn.Linear(flattened_img_size, 512),
    nn.Dropout(.3),
    nn.ReLU(),
    nn.Linear(512, n_classes)
)

### 3.3.1 [Conv2D](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html)

<img src="images/conv2d.png" width=300 />

這些是我們的 2D 卷積層(Conv2D)。許多小型核心(kernel)會掃過輸入圖像並檢測對分類重要的特徵。模型中較早的卷積會檢測簡單的特徵，如線條。較後的卷積則會檢測更複雜的特徵。讓我們看看我們的第一個 Conv2D 層：
```Python
nn.Conv2d(IMG_CHS, 25, kernel_size, stride=1, padding=1)
```
25 指的是將要學習的過濾器(filter)數量。雖然 `kernel_size = 3`，PyTorch 會假設我們想要 3 x 3 的過濾器。步長(stride)指的是過濾器掃過圖像時的步長大小。填充(padding)代表的是由過濾器創建的輸出圖像的大小是否與輸入圖像大小一致。

### 3.3.2 [BatchNormalization](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html)



就像正規化(Normalization)我們的輸入的資料一樣，批次正規化(BatchNormalization)會調整隱藏層中的值以改善訓練。[在這裡詳細了解更多](https://blog.paperspace.com/busting-the-myths-about-batch-normalization/)。

關於批次正規化層(BatchNormalization)的最佳放置位置存在爭議。這個 [Stack Overflow](https://stackoverflow.com/questions/39691902/ordering-of-batch-normalization-and-dropout) 貼文彙整了許多觀點。

### 3.3.3 [MaxPool2D](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html)

<img src="images/maxpool2d.png" width=300 />

最大池化(MaxPool2D)會取一個圖像並基本上將其縮小到較低的解析度。這樣做是為了幫助模型對橫移（物體左右移動）具有強韌性(robust)，同時也使我們的模型更快速。

### 3.3.4 [Dropout](https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html)

<img src="images/dropout.png" width=360 />

 Dropout 是一種防止過度擬合(overfitting)的技術。Dropout 隨機選擇一部分神經元並將其關閉，使它們在特定的前向或後向傳播中不參與。這有助於確保網路具有強韌性和備援性(redundant)，並且不依賴於任何一個區域來得出答案。

### 3.3.5 [Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html)



Flatten 將一層的多維輸出展平成一維陣列。輸出被稱為特徵向量(feature vector)，並將連接到最終的分類層。

### 3.3.6 [Linear](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)


我們在先前的模型中已經見過密集線性層(dense linear layers)。我們的第一個密集層（512 個單元）將特徵向量作為輸入，並學習哪些特徵會對特定分類有所貢獻。第二個密集層（24 個單元）是最終的分類層，輸出我們的預測結果。

## 3.4 Summarizing the Model


這可能感覺像是很多資訊，但別擔心。現在不需要完全理解所有內容就能有效地訓練卷積模型。最重要的是我們知道它們可以幫助從圖像中提取有用的資訊，並可用於分類任務。

In [14]:
model = torch.compile(model.to(device))
model

OptimizedModule(
  (_orig_mod): Sequential(
    (0): Conv2d(1, 25, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(25, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.2, inplace=False)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(50, 75, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): BatchNorm2d(75, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (13): Flatten(start_dim=1, end_dim=-1)
    (14): Linear(in_features=675, out_features=512, bias=True)
    (15): Dropout


由於我們嘗試解決的問題仍然相同（分類 ASL 圖像），我們將繼續使用相同的 `loss_function` 和  `accuracy` 指標。

In [15]:
loss_function = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters())

In [16]:
def get_batch_accuracy(output, y, N):
    pred = output.argmax(dim=1, keepdim=True)
    correct = pred.eq(y.view_as(pred)).sum().item()
    return correct / N

### 3.5 訓練模型

儘管模型架構非常不同，但訓練過程看起來完全相同。


#### 實作練習


這些是與之前相同的 `train` 和 `validate` 函式，但它們已被混合。你能正確命名每個函式並替換 `FIXME` 嗎？

其中一個應該有 `model.train`，另一個應該有 `model.eval`。

In [17]:
def FIXME():
    loss = 0
    accuracy = 0

    model.FIXME()
    with torch.no_grad():
        for x, y in FIXME:
            output = model(x)

            loss += loss_function(output, y).item()
            accuracy += get_batch_accuracy(output, y, valid_N)
    print('FIXME - Loss: {:.4f} Accuracy: {:.4f}'.format(loss, accuracy))

In [18]:
def FIXME():
    loss = 0
    accuracy = 0

    model.FIXME()
    for x, y in FIXME:
        output = model(x)
        optimizer.zero_grad()
        batch_loss = loss_function(output, y)
        batch_loss.backward()
        optimizer.step()

        loss += batch_loss.item()
        accuracy += get_batch_accuracy(output, y, train_N)
    print('FIXME - Loss: {:.4f} Accuracy: {:.4f}'.format(loss, accuracy))

#### 解答


點擊下方兩個 `...` 查看解答。

In [19]:
# SOLUTION
def validate():
    loss = 0
    accuracy = 0

    model.eval()
    with torch.no_grad():
        for x, y in valid_loader:
            output = model(x)

            loss += loss_function(output, y).item()
            accuracy += get_batch_accuracy(output, y, valid_N)
    print('Valid - Loss: {:.4f} Accuracy: {:.4f}'.format(loss, accuracy))

In [20]:
# SOLUTION
def train():
    loss = 0
    accuracy = 0

    model.train()
    for x, y in train_loader:
        output = model(x)
        optimizer.zero_grad()
        batch_loss = loss_function(output, y)
        batch_loss.backward()
        optimizer.step()

        loss += batch_loss.item()
        accuracy += get_batch_accuracy(output, y, train_N)
    print('Train - Loss: {:.4f} Accuracy: {:.4f}'.format(loss, accuracy))

In [21]:
epochs = 20

for epoch in range(epochs):
    print('Epoch: {}'.format(epoch))
    train()
    validate()

Epoch: 0
Train - Loss: 265.9026 Accuracy: 0.9071
Valid - Loss: 33.6062 Accuracy: 0.9506
Epoch: 1
Train - Loss: 15.6663 Accuracy: 0.9950
Valid - Loss: 20.6965 Accuracy: 0.9672
Epoch: 2
Train - Loss: 14.7827 Accuracy: 0.9952
Valid - Loss: 16.0878 Accuracy: 0.9647
Epoch: 3
Train - Loss: 9.8377 Accuracy: 0.9965
Valid - Loss: 14.8248 Accuracy: 0.9738
Epoch: 4
Train - Loss: 11.2571 Accuracy: 0.9958
Valid - Loss: 15.5076 Accuracy: 0.9710
Epoch: 5
Train - Loss: 4.2680 Accuracy: 0.9987
Valid - Loss: 14.3364 Accuracy: 0.9739
Epoch: 6
Train - Loss: 7.9964 Accuracy: 0.9972
Valid - Loss: 15.4875 Accuracy: 0.9808
Epoch: 7
Train - Loss: 11.2621 Accuracy: 0.9961
Valid - Loss: 10.4677 Accuracy: 0.9796
Epoch: 8
Train - Loss: 0.4117 Accuracy: 0.9999
Valid - Loss: 7.8629 Accuracy: 0.9847
Epoch: 9
Train - Loss: 0.1026 Accuracy: 1.0000
Valid - Loss: 9.1475 Accuracy: 0.9848
Epoch: 10
Train - Loss: 13.4166 Accuracy: 0.9956
Valid - Loss: 17.3810 Accuracy: 0.9777
Epoch: 11
Train - Loss: 1.6007 Accuracy: 0.9995


### 3.5.1 結果討論



看起來這個模型有顯著的改進！訓練準確度(accuracy)非常高，驗證準確度(validation accuracy)也有所提高。這是一個很好的結果，我們只需要換一個新模型就做到了。

你可能已經注意到驗證準確度(validation accuracy)在跳動。這表明我們的模型仍然沒有完美地泛化(generalizing)。幸運的是，我們還可以做更多事情。讓我們在下一個講座中討論它。

## 3.6 總結



在本節中，我們使用了幾種新型的層來實現卷積神經網路(CNN)，它比上一節使用的簡單的模型表現得更好。希望你開始對使用準備好的資料來建立和訓練模型的整體過程越來越熟悉。

### 3.6.1 清空記憶體


在繼續之前，請執行以下程式碼區塊(Cell)以清空 GPU 記憶體。這是繼續下一個notebook所必需的。

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### 3.6.2 下一步



在過去的幾個部分中，你專注於模型的創建和訓練。為了進一步提高性能，你現在將注意力轉向資料增強(data augmentation)，這是一系列技術，將允許你的模型在比原始可用資料更多更好的資料上進行訓練。

<center><a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a></center>